In [21]:
import pandas as pd
import os
import logging

In [22]:
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

In [23]:
baojiadan_folder = 'D:\\Program Files (x86)\\百度云同步盘\\Dropbox\\-E·J- 2014.5.1\\2015.9.26 燕文\\'

In [24]:
baojiadan_file_list = list()

In [25]:
for file_name in os.listdir(baojiadan_folder):
    if file_name.startswith('广州燕文线下报价单'):
        baojiadan_file_list.append(file_name)

In [26]:
latest_baojiadan_file_name = baojiadan_file_list[-1]
# latest_baojiadan_file_name

In [27]:
latest_baojiadan_path = os.path.join(baojiadan_folder, latest_baojiadan_file_name)
# latest_baojiadan_path

In [28]:
baojiadan_sheets_dict = pd.read_excel(latest_baojiadan_path, sheetname=None)
# type(baojiadan)

In [29]:
wanted_shipping_service_list = [
                                '燕文航空经济小包(普货)',
                                '燕文航空经济小包-特货',
                                '燕文专线平邮小包-普货',
                                '燕文专线平邮小包-特货',
                                '燕邮宝平邮Mini-特货',
                                '燕邮宝平邮-普货',
                                '燕邮宝平邮-特货',
                                '燕文英国快线平邮-普货',
                                '燕文英国快线平邮-特货'

]

In [30]:
export_path = os.path.join(baojiadan_folder, '报价单','燕文报价表格（平邮）.xlsx')
writer = pd.ExcelWriter(export_path)

pd_to_merge = []

for service_name in wanted_shipping_service_list:
    if service_name in baojiadan_sheets_dict.keys():
        logging.info(service_name + ' is ready to export.')
        df = baojiadan_sheets_dict[service_name]
        try:
            for c in df.columns:
                for index, r in enumerate(df[c]):
                    if r == '国家':
                        country_cell_location = [index, c]
#                         print(country_cell_location)
                    if r == '价格使用说明:':
                        end_location = [index, c]
#                         print(country_cell_location)
                        break
            df = df[country_cell_location[0]+1:end_location[0]]
            df = df.dropna(how='all')
            df = df[df[country_cell_location[1]].str.len() < 10]
            df['service_name'] = service_name
#             print(df.head(5))
            
#             tiers = [1, 2, 3]
#             for tier in tiers:
#                 this_c_name = 'tier_' + str(tier)
#                 next_c = tier + 1
#                 try:
#                     df[this_c_name + '_base'] = df[[tier]]
#                     df[this_c_name + '_surcharge'] = df[[next_c]]
#                     df[this_c_name + '_total'] = df[[tier]].astype(float) * 0.35 + df[this_c_name + '_surcharge'].astype(float)
#                 except Exception as err:
#                     print(str(err))
            
            kg = 0.35
            tiers = [1, 2, 3]
            for tier in tiers:
                this_c_name = 'tier_' + str(tier)
                column_num = 2 * tier - 1
                try:
                    df[this_c_name + '_kg_fee'] = df[[column_num]] * kg
                    df[this_c_name + '_kg_fee'] = df[this_c_name + '_kg_fee'].astype(float)
                    df[this_c_name + '_surcharge'] = df[[column_num + 1]].astype(float)
                    df[this_c_name + '_total'] = df[this_c_name + '_kg_fee'] + df[this_c_name + '_surcharge']
                    max_tier = tier
                except Exception as err:
                    print(str(err))
                    max_tier = tier - 1
                    break
            logging.info('max tier for ' + service_name + ' is ' + str(max_tier))

            # 某几个特殊渠道，要修改max_tier
            if service_name in ('燕文C优先小包',
                                '中邮深圳线下E邮宝-普货',
                                '中邮深圳线下E邮宝-特货',
                                '中邮广州E邮宝(线下)',
                                '湖南E邮宝',
                                '中邮郑州线下E邮宝',
                                '中邮新疆E邮宝(线下)',
                                '中邮北京中俄陆运E邮宝'):
                max_tier = 1
            
            for_merge_tier_kg_fee = 'tier_' + str(max_tier) + '_kg_fee'
            for_merge_tier_surcharge = 'tier_' + str(max_tier) + '_surcharge'
            for_merge_tier_total = 'tier_' + str(max_tier) + '_total'
            
            df_for_merge = df[['service_name', service_name, for_merge_tier_kg_fee, for_merge_tier_surcharge, for_merge_tier_total]]
            df_for_merge = df_for_merge.rename(columns={df_for_merge.columns[1]: "country",
                                                        df_for_merge.columns[2]: "kg_fee",
                                                        df_for_merge.columns[3]: "surcharge", 
                                                        df_for_merge.columns[4]: "total"})
            
            # 中邮渠道需要乘以折扣
            service_discount = 0.93
            if service_name in ('中邮深圳挂号小包'):
                df_for_merge['total'] = df_for_merge['total'] * service_discount
            
            pd_to_merge.append(df_for_merge)
    
            df.to_excel(writer, sheet_name=service_name, index=False)
            logging.info(service_name + ' exported.')
            # df.to_csv(os.path.join(baojiadan_folder, '报价单', key + '.csv'))
        except Exception as err:
            logging.info(service_name + ' failed to export. ' + str(err))
            pass
        # df.to_csv(os.path.join(baojiadan_folder, '报价单', key + '.xlsx'))
        
writer.save()

# 合并计算结果
final_merger = pd.concat(pd_to_merge).reset_index()

# 检查一个渠道对同一个国家有多少种计算结果
final_merger.groupby(['service_name', 'country']).agg({'total': 'count'}).reset_index().to_csv(os.path.join(baojiadan_folder, '报价单', '燕文线下发货渠道国家表（平邮）.csv'))

# 分组排名
final_merger['rn'] = final_merger.sort_values(['total'], ascending=[True]).groupby(['country']).cumcount() + 1
final_merger = final_merger.sort_values(['country', 'rn'], ascending=[True, True])
final_merger = final_merger[final_merger['rn'] <= 3]
final_merger.to_csv(os.path.join(baojiadan_folder, '报价单', '燕文线下发货合并表（平邮）.csv'))

final_merger = final_merger[final_merger['rn'] == 1]
final_merger.sort_values(['service_name', 'total'], ascending=[True, True]).to_csv(os.path.join(baojiadan_folder, '报价单', '燕文线下发货合并表（最优解）（平邮）.csv'))

 2020-04-08 00:28:19,016 - INFO - 燕文航空经济小包(普货) is ready to export.
 2020-04-08 00:28:19,077 - INFO - max tier for 燕文航空经济小包(普货) is 2
 2020-04-08 00:28:19,135 - INFO - 燕文航空经济小包(普货) exported.
 2020-04-08 00:28:19,135 - INFO - 燕文航空经济小包-特货 is ready to export.
 2020-04-08 00:28:19,179 - INFO - max tier for 燕文航空经济小包-特货 is 2
 2020-04-08 00:28:19,225 - INFO - 燕文航空经济小包-特货 exported.
 2020-04-08 00:28:19,226 - INFO - 燕文专线平邮小包-普货 is ready to export.
 2020-04-08 00:28:19,236 - INFO - max tier for 燕文专线平邮小包-普货 is 1
 2020-04-08 00:28:19,248 - INFO - 燕文专线平邮小包-普货 exported.
 2020-04-08 00:28:19,249 - INFO - 燕文专线平邮小包-特货 is ready to export.
 2020-04-08 00:28:19,259 - INFO - max tier for 燕文专线平邮小包-特货 is 1
 2020-04-08 00:28:19,268 - INFO - 燕文专线平邮小包-特货 exported.
 2020-04-08 00:28:19,269 - INFO - 燕邮宝平邮Mini-特货 is ready to export.
 2020-04-08 00:28:19,321 - INFO - max tier for 燕邮宝平邮Mini-特货 is 1


could not convert string to float: '燕文航空经济小包(普货)'
could not convert string to float: '燕文航空经济小包-特货'
Could not operate 0.35 with block values can't multiply sequence by non-int of type 'float'
Could not operate 0.35 with block values can't multiply sequence by non-int of type 'float'
could not convert string to float: '燕邮宝平邮Mini-特货'


 2020-04-08 00:28:19,327 - INFO - 燕邮宝平邮Mini-特货 exported.
 2020-04-08 00:28:19,328 - INFO - 燕邮宝平邮-普货 is ready to export.
 2020-04-08 00:28:19,338 - INFO - max tier for 燕邮宝平邮-普货 is 1
 2020-04-08 00:28:19,349 - INFO - 燕邮宝平邮-普货 exported.
 2020-04-08 00:28:19,349 - INFO - 燕邮宝平邮-特货 is ready to export.
 2020-04-08 00:28:19,363 - INFO - max tier for 燕邮宝平邮-特货 is 1
 2020-04-08 00:28:19,374 - INFO - 燕邮宝平邮-特货 exported.
 2020-04-08 00:28:19,375 - INFO - 燕文英国快线平邮-普货 is ready to export.
 2020-04-08 00:28:19,425 - INFO - max tier for 燕文英国快线平邮-普货 is 2
 2020-04-08 00:28:19,429 - INFO - 燕文英国快线平邮-普货 exported.
 2020-04-08 00:28:19,430 - INFO - 燕文英国快线平邮-特货 is ready to export.
 2020-04-08 00:28:19,478 - INFO - max tier for 燕文英国快线平邮-特货 is 2
 2020-04-08 00:28:19,482 - INFO - 燕文英国快线平邮-特货 exported.


Could not operate 0.35 with block values can't multiply sequence by non-int of type 'float'
Could not operate 0.35 with block values can't multiply sequence by non-int of type 'float'
could not convert string to float: '燕文英国快线平邮-普货'
could not convert string to float: '燕文英国快线平邮-特货'
